In [1]:
import os
import sys

import matplotlib.pyplot as plt
import numpy
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.datasets import mnist, fashion_mnist
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential

# seed 값 설정
# seed = 0
# numpy.random.seed(seed)
# tf.random.set_seed(3)

def loadData():
    # MNIST데이터셋 불러오기
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    print("학습셋 shape", end=':')
    print(x_train.shape)
    print("학습셋: {0}개, 테스트셋: {1}개".format(x_train.shape[0], x_test.shape[0]))
    return (x_train, y_train), (x_test, y_test)


def drawImage(data):
    plt.imshow(data, cmap='Greys')
    plt.show()

def printImage(data):
    for x in data:
        for i in x:
            sys.stdout.write('%d\t' % i)
        sys.stdout.write('\n')

def reshape2DTo1D(arr):
    #(60000, 28, 28) -> (60000, 784)
    reshaped_arr = arr.reshape(arr.shape[0], 784)
    scaled_arr = reshaped_arr / 255
    return scaled_arr

def reshape2DTo3D(arr):
    #(60000, 28, 28) -> (60000, 28, 28, 1)
    scaled_arr = arr.reshape(arr.shape[0], 28, 28, 1) / 255
    return scaled_arr
    #X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
    # X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255

def createDenseModel():
    # 모델 프레임 설정
    model = Sequential()
    model.add(Dense(512, input_dim=784, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    return model

def createCnnModel():
    # 컨볼루션 신경망의 설정
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    return model

def trainModel(model, X_train, Y_train, X_test, Y_test):
    # 모델 실행 환경 설정
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    # 모델 최적화 설정
    MODEL_DIR = './model/'
    if not os.path.exists(MODEL_DIR):
        os.mkdir(MODEL_DIR)

    modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
    checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

    # 모델의 실행
    history = model.fit(X_train, Y_train,
                        validation_data=(X_test, Y_test),
                        epochs=30, batch_size=200, verbose=0,
                        callbacks=[early_stopping_callback, checkpointer])

    return model, history

def drawLossChart(history):
    # 테스트 셋의 오차
    y_vloss = history.history['val_loss']

    # 학습셋의 오차
    y_loss = history.history['loss']

    # 그래프로 표현
    x_len = numpy.arange(len(y_loss))
    plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
    plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

    # 그래프에 그리드를 주고 레이블을 표시
    plt.legend(loc='upper right')
    # plt.axis([0, 20, 0, 0.35])
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

#(60000, 28, 28)
(x_train, y_train), (x_test, y_test) = loadData()
# 그래프로 확인
drawImage(x_train[0])
printImage(x_train[0])
print("class:{}".format(y_train[0]))

cnn = True

if(cnn):
    # (60000, 28, 28)
    x_train = reshape2DTo3D(x_train)
    x_test = reshape2DTo3D(x_test)
else:
    # (60000, 784)
    x_train = reshape2DTo1D(x_train)
    x_test = reshape2DTo1D(x_test)

x_train.shape
# print(x_train[0])

# 바이너리화 과정
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print(y_train[0])

if(cnn):
    model = createCnnModel()
else:
    model = createDenseModel()

model.summary()
model_trained, history = trainModel(model, x_train, y_train, x_test, y_test)
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

drawLossChart(history)


ModuleNotFoundError: No module named 'tensorflow'